The __tangle__ step in literate programming converts text in the specification language to the source language; the source language must be valid to the object language.  [`pidgin.tangle`](tangle.ipynb) transpiles __Markdown__ to valid `IPython` flavored __Python__ using the `IPython` configuration system.

Literate `pidgin` documents are reusable as __Python__ modules, documentation, and tests.

## IPython configuration.

In [1]:
    import importnb.utils.pytest_importnb

In [2]:
    import ast, IPython, importnb, nbformat
    with importnb.Notebook():
        from pidgin import tangle

In [3]:
    __all__ = "PidginMarkdown ParameterizeMarkdown PidginWeave PidginParameterize".split()#ParameterizeWeave

`MarkdownImporter` imports __Markdown__ files are source.  By default they recieve a Markdown repr.

A markdown file is a notebook with one cell.

In [4]:
    def visit(PidginMixin, node): 
        from ..specifications import json, emojis, yaml, transform_ast
        """These have to be applied outer most first."""
        for visitor in reversed((
            yaml.Yaml, json.Expand, emojis.Emojize, transform_ast.ReturnYieldDisplay, json.Pointer, 
        )): node = visitor().visit(node)
        return node
    tangle.PidginMixin.visit = visit

In [5]:
    class MarkdownMixin:
        extensions = '.md',
        def get_data(MarkdownMixin, path): return MarkdownMixin.code(MarkdownMixin.decode())
        def exec_module(MarkdownMixin, module):
            super().exec_module(module)
            module._ipython_display_ = lambda: IPython.display.display(IPython.display.Markdown(filename=module.__file__))

In [6]:
    class MarkdownImporter(MarkdownMixin, tangle.PidginMixin, importnb.Notebook):  ...

In [7]:
    class PidginMarkdown(MarkdownMixin, tangle.PidginMixin, importnb.Notebook):  ...

# Parameterized loader

In [8]:
    class ParameterizeMarkdown(MarkdownMixin, tangle.PidginMixin, importnb.Parameterize):  ...

In [9]:
    class PidginParameterize(tangle.PidginMixin, importnb.Parameterize): ...

Command line

In [10]:
    if importnb.ipython_extension.IPYTHON_MAIN():
        Pidgin, Markdown = PidginParameterize, ParameterizeMarkdown
    else: Markdown =  MarkdownImporter

# Implicit loader

In [11]:
    class PidginWeave(tangle.Pidgin): 
        __slots__ = importnb.Notebook.__slots__ + ('_weave', )
        def __init__(self, *args, weave=True, **kwargs):        
            super().__init__(*args, **kwargs)
            self._weave = weave
            
        def exec_module(Pidgin, module):
            super().exec_module(module)
            with tangle.Pidgin(): from pidgin import weave, testing
            if Pidgin._weave:
                shell = weave.Static(
                    parent=IPython.get_ipython(), user_module=module, parents=[
                        testing.Doctest(user_module=module, parent=IPython.get_ipython()), weave.Markdown(user_module=module, parent=IPython.get_ipython())
                    ]
                )
                with IPython.utils.capture.capture_output(stderr=False, stdout=False) as out, open(module.__file__) as f: 
                    return [shell.run_cell(object['source']) for object in nbformat.read(f, 4)['cells']
                         if object['cell_type'] in {"code", "markdown"}]
                module._ipython_display_ = lambda:out.show()

